# 1. PATH CONFIGURATION

In [1]:
import os
import json
import glob
import re
import pandas as pd

# --- 1. PATH CONFIGURATION ---

# Current working directory (where this Notebook is located)
CURRENT_DIR = os.getcwd()

# Move one level up (..) to access the Raw_Data_Set folder
RAW_DATA_DIR = os.path.join(CURRENT_DIR, '..', '1. Raw_Data_Set')

# Define exact paths for data sources
PATH_LINKEDIN = os.path.join(RAW_DATA_DIR, 'LinkedIn')
PATH_GLASSDOOR = os.path.join(RAW_DATA_DIR, 'GlassDoor')

# Configuration files (located in the same folder as this Notebook)
FILE_ROLES = 'SOC_Analyst_Roles.json'
FILE_SKILLS = 'Skills.json'
FILE_LOCATIONS_FIX = 'Locations.json'

# Final output file for Tableau
FINAL_OUTPUT = 'SOC_Analyst_Master_Dataset.json'

print(f"📂 Working Directory: {CURRENT_DIR}")
print(f"📂 Raw Data Directory detected at: {os.path.abspath(RAW_DATA_DIR)}")

📂 Working Directory: C:\Users\iurie\3. Tableau Projects\01. SOC-Analyst-Job-Market-Analysis\2. Data_Processing_Process
📂 Raw Data Directory detected at: C:\Users\iurie\3. Tableau Projects\01. SOC-Analyst-Job-Market-Analysis\1. Raw_Data_Set


# 2. LOAD CONFIGURATION FILES

In [2]:
# --- 2. LOAD CONFIGURATION FILES ---

try:
    # Load Roles Definition
    with open(FILE_ROLES, 'r', encoding='utf-8') as f:
        roles_data = json.load(f)
        # Handle cases where data is nested under "all roles SOC"
        ROLES_DEF = roles_data.get("all roles SOC", roles_data)

    # Load Skills Definition
    with open(FILE_SKILLS, 'r', encoding='utf-8') as f:
        skills_data = json.load(f)
        SKILLS_DEF = skills_data.get("Skills", skills_data)

    # Load Location Fixes (Mapping)
    with open(FILE_LOCATIONS_FIX, 'r', encoding='utf-8') as f:
        loc_list = json.load(f)
        # Convert list to dictionary for O(1) lookup speed using 'location' as key
        LOC_FIX_MAP = {item['location']: item for item in loc_list}

    print("✅ Configuration files (Roles, Skills, Locations) loaded successfully.")

except FileNotFoundError as e:
    print(f"❌ ERROR: Configuration file not found: {e}")
except Exception as e:
    print(f"❌ UNEXPECTED ERROR while reading config: {e}")

✅ Configuration files (Roles, Skills, Locations) loaded successfully.


# 3. DATA NORMALIZATION FUNCTION

In [3]:
# --- 3. DATA NORMALIZATION FUNCTION (FIXED: Added missing fields) ---

def normalize_job_structure(job_data, source_name):
    """
    Accepts a raw job object (dict) and returns a standardized dictionary.
    Includes experienceLevel and companyUrl to match the old dataset structure.
    """
    normalized = {}
    
    if source_name == "LinkedIn":
        normalized = {
            "title": job_data.get("title"),
            "companyName": job_data.get("companyName"),
            "location": job_data.get("location"),
            "jobUrl": job_data.get("jobUrl"),
            "contractType": job_data.get("contractType"),
            "sector": job_data.get("sector"),
            # CÂMPURI ADĂUGATE:
            "experienceLevel": job_data.get("experienceLevel"),
            "companyUrl": job_data.get("companyUrl"),
            
            "description": job_data.get("description"),
            "source": "LinkedIn"
        }
    elif source_name == "GlassDoor":
        # Glassdoor location cleaning
        loc_obj = job_data.get("job_location")
        loc_str = ""
        if isinstance(loc_obj, dict):
            parts = [p for p in [loc_obj.get("city"), loc_obj.get("country")] if p]
            loc_str = ", ".join(parts)
        elif isinstance(loc_obj, str):
            loc_str = loc_obj

        normalized = {
            "title": job_data.get("job_title"),
            "companyName": job_data.get("company_name"),
            "location": loc_str,
            "jobUrl": job_data.get("job_url"),
            "contractType": job_data.get("job_job_types"),
            "sector": job_data.get("job_industry"),
            # CÂMPURI ADĂUGATE (Glassdoor le numește altfel uneori, folosim get safe):
            "experienceLevel": job_data.get("job_levels"), 
            "companyUrl": job_data.get("company_url"), 
            
            "description": job_data.get("job_description"),
            "source": "GlassDoor"
        }
    
    # Basic string cleaning
    if normalized['location']:
        normalized['location'] = normalized['location'].strip()
        
    return normalized

# 4. DATA PROCESSING PIPELINE (ETL)

In [4]:
# --- 4. DATA PROCESSING PIPELINE (ETL) - ADVANCED LOCATION FIX ---

all_jobs = []
processed_jobs = []
seen_ids = set() 

# Statistics
stats = {
    "total_read": 0,
    "excluded_filter": 0,
    "excluded_duplicate": 0,
    "final_count": 0
}

# Keywords
KEYWORDS_FILTER = ["soc", "security operation", "incident response", "threat intelligence", "cyber defense", "siem", "blue team"]

# --- MAPARE MANUALĂ PENTRU ZONE METROPOLITANE (FIX PENTRU CELE 52 TĂRI) ---
# Aici transformăm "Regiunile" care apar ca țări în Țara corectă.
REGION_TO_COUNTRY_MAP = {
    "Stuttgart Region": "Germany",
    "Greater Milan Metropolitan Area": "Italy",
    "Greater Hamburg Area": "Germany",
    "Copenhagen Metropolitan Area": "Denmark",
    "Timisoara Metropolitan Area": "Romania",
    "Sofia Metropolitan Area": "Bulgaria",
    "Lodz Metropolitan Area": "Poland",
    "Katowice Metropolitan Area": "Poland",
    "Greater Zaragoza Metropolitan Area": "Spain",
    "Greater Turin Metropolitan Area": "Italy",
    "Greater Toulouse Metropolitan Area": "France",
    "Greater Lyon Area": "France",
    "Greater Leipzig Area": "Germany",
    "Greater Kiel Area": "Germany",
    "Greater Helsingborg Metropolitan Area": "Sweden",
    "Greater Granada Metropolitan Area": "Spain",
    "Greater Bilbao Metropolitan Area": "Spain",
    "Greater Vasteras Metropolitan Area": "Sweden",
    "Greater Munchen Metropolitan Area": "Germany",
    "Greater Paris Metropolitan Area": "France",
    "Greater London": "United Kingdom",
    "Randstad": "Netherlands",
    "Ile-De-France": "France"
}

print("🚀 Starting Data Processing Pipeline...")

# --- STEP A: INGESTION ---
files_list = []
files_list.extend(glob.glob(os.path.join(PATH_LINKEDIN, "*.json")))
files_list.extend(glob.glob(os.path.join(PATH_GLASSDOOR, "*.json")))

print(f"📂 Found {len(files_list)} JSON files in total.")

for file in files_list:
    try:
        with open(file, 'r', encoding='utf-8') as f:
            data = json.load(f)
            source = "LinkedIn" if "LinkedIn" in file else "GlassDoor"
            if isinstance(data, list):
                for job in data:
                    all_jobs.append(normalize_job_structure(job, source))
            elif isinstance(data, dict):
                 all_jobs.append(normalize_job_structure(data, source))
    except Exception as e:
        print(f"⚠️ Error reading file {os.path.basename(file)}: {e}")

stats["total_read"] = len(all_jobs)
print(f"📊 Total raw jobs extracted: {stats['total_read']}")


# --- STEP B: TRANSFORMATION ---

for job in all_jobs:
    title = str(job.get("title", "")).lower()
    desc = str(job.get("description", "")).lower()
    
    # 1. FILTER
    is_relevant = any(kw in title for kw in KEYWORDS_FILTER) or any(kw in desc for kw in KEYWORDS_FILTER)
    if not is_relevant:
        stats["excluded_filter"] += 1
        continue 

    # 2. DEDUPLICATION
    comp = str(job.get("companyName", "")).lower().replace(" ", "").replace(",", "")
    loc = str(job.get("location", "")).lower().replace(" ", "").replace(",", "")
    tit_simple = title.replace(" ", "")
    unique_id = f"{tit_simple}|{comp}|{loc}"
    if unique_id in seen_ids:
        stats["excluded_duplicate"] += 1
        continue
    seen_ids.add(unique_id)

    # 3. SECTOR CLEANING
    raw_sector = job.get("sector")
    if not raw_sector or str(raw_sector).strip() == "":
        job['sector'] = "Not Specified"
    else:
        job['sector'] = str(raw_sector).strip()

    # 4. GEOGRAPHIC PARSING (REWORKED)
    job['city'] = None
    job['region'] = None
    job['country'] = None
    
    raw_loc = job.get("location", "")
    
    if raw_loc:
        parts = [p.strip() for p in raw_loc.split(',')]
        
        # LOGICA NOUĂ:
        
        # CAZ 3 ELEMENTE: "City, Region, Country"
        if len(parts) >= 3:
            job['city'] = parts[0]
            job['region'] = ", ".join(parts[1:-1])
            job['country'] = parts[-1].title()

        # CAZ 2 ELEMENTE: "City, Country" (ex: "Gloucester, United Kingdom")
        elif len(parts) == 2:
            job['city'] = parts[0]
            job['country'] = parts[1].title() # Al doilea element e țara
            
        # CAZ 1 ELEMENT: "Country" SAU "Metropolitan Area" (ex: "Stuttgart Region")
        elif len(parts) == 1:
            candidate = parts[0]
            # Verificăm dacă e în lista noastră de corecții (Metropolitan Areas)
            # Folosim o căutare parțială (dacă string-ul conține cheia) sau exactă
            match_found = False
            for region_key, correct_country in REGION_TO_COUNTRY_MAP.items():
                if region_key.lower() in candidate.lower():
                    job['country'] = correct_country
                    job['region'] = candidate # Păstrăm originalul la regiune
                    match_found = True
                    break
            
            if not match_found:
                # Dacă nu e în lista de corecții, presupunem că e Țară
                job['country'] = candidate.title()

    # CURĂȚARE FINALĂ ȚARĂ & ORAȘ
    # 1. Dacă țara a rămas goală, punem "Unknown"
    if not job['country']:
         job['country'] = "Unknown"
         
    # 2. Dacă Orașul == Țara (ex: "United Kingdom, United Kingdom"), ștergem orașul
    if job['city'] and job['country'] and job['city'].lower().strip() == job['country'].lower().strip():
        job['city'] = None

    # 5. LOCATION FIXES (From Locations.json)
    if job.get('region') is None and job.get('location') in LOC_FIX_MAP:
        fix_data = LOC_FIX_MAP[job['location']]
        job['country'] = fix_data.get('country')
        job['region'] = fix_data.get('region')
        job['city'] = fix_data.get('city')

    # 6. ROLE ASSIGNMENT
    assigned_role = "Uncategorized"
    extended_roles = {
        "Direct SOC Analyst": ["soc", "security analyst", "analyst", "operator", "monitoring", "intern", "student", "trainee", "junior", "graduate", "stage", "werkstudent", "watch", "handler", "analyste", "duales", "placement", "apprentice"],
        "SOC Technology Specialists": ["engineer", "network", "system", "admin", "integrator", "architect", "support", "technician", "embedded", "infrastructure", "developer", "iam", "pki", "access", "identity", "cloud", "devops", "platform", "pentest", "hacker", "offensive", "vulnerability", "tester", "spezialist", "specialist", "expert", "sicherheit", "security professional", "engineer", "ingenieur", "product owner", "scientist"],
        "Incident Response and Threat Intelligence": ["incident", "response", "threat", "intelligence", "forensic", "hunter", "cert", "csirt", "ir", "malware", "investigator", "crypto"],
        "Sales, Management, Training & Consulting": ["manager", "head", "lead", "sales", "consultant", "trainer", "director", "coordinator", "officer", "ciso", "presales", "account", "risk", "compliance", "audit", "governance", "awareness", "legal", "advisor", "leiter", "vp", "vice president", "president", "professor", "lecturer", "teacher", "academic", "research", "advocate", "claims"],
        "Cyber Defense and Operations": ["defense", "defence", "blue", "operations", "ops", "secops", "protect", "resilience"]
    }
    match_found = False
    for role_name, keywords in ROLES_DEF.items():
        if any(k.lower() in title for k in keywords):
            assigned_role = role_name
            match_found = True
            break
    if not match_found:
        for role_name, keywords in extended_roles.items():
            if any(k in title for k in keywords):
                assigned_role = role_name
                break
    job['role'] = assigned_role

    # 7. SKILLS EXTRACTION
    job_skills = {}
    for category, skill_list in SKILLS_DEF.items():
        found_skills = []
        for skill in skill_list:
            if re.search(r'\b' + re.escape(skill) + r'\b', desc, re.IGNORECASE):
                found_skills.append(skill)
        if found_skills:
            job_skills[category] = found_skills
    job['skills'] = job_skills

    # FINAL ORDERING
    ordered_job = {
        "title": job.get("title"),
        "companyName": job.get("companyName"),
        "location": job.get("location"),
        "country": job.get("country"),
        "region": job.get("region"),
        "city": job.get("city"),
        "jobUrl": job.get("jobUrl"),
        "contractType": job.get("contractType"),
        "sector": job.get("sector"),
        "experienceLevel": job.get("experienceLevel"),
        "companyUrl": job.get("companyUrl"),
        "description": job.get("description"),
        "source": job.get("source"),
        "role": job.get("role"),
        "skills": job.get("skills")
    }

    processed_jobs.append(ordered_job)

stats["final_count"] = len(processed_jobs)

print("\n--- FINAL EXECUTION REPORT ---")
print(f"1. Total Jobs Scanned: {stats['total_read']}")
print(f"2. Excluded (Irrelevant): -{stats['excluded_filter']}")
print(f"3. Excluded (Duplicates): -{stats['excluded_duplicate']}")
print(f"==========================================")
print(f"✅ FINAL DATASET FOR TABLEAU: {stats['final_count']} jobs")

🚀 Starting Data Processing Pipeline...
📂 Found 41 JSON files in total.
📊 Total raw jobs extracted: 4076

--- FINAL EXECUTION REPORT ---
1. Total Jobs Scanned: 4076
2. Excluded (Irrelevant): -1678
3. Excluded (Duplicates): -61
✅ FINAL DATASET FOR TABLEAU: 2337 jobs


# 5. CHECKING DATA SET

In [5]:
# --- 5. CHECKING DATA SET (Quality Assurance) ---

import pandas as pd
from IPython.display import display # Import necessary for nice HTML tables

print("🔍 Starting Data Quality Check...")

# Convert the list of processed jobs into a Pandas DataFrame
df = pd.DataFrame(processed_jobs)

# 1. Analyze Job Role Distribution
print("\n--- 📊 Job Role Distribution ---")
print(df['role'].value_counts())

# 2. Visual Inspection (HTML Table)
print("\n--- 🎲 Random Sample of 5 Jobs (Clean Data Preview) ---")

# Setup generic pandas display options to ensure text isn't cut off too aggressively
pd.set_option('display.max_colwidth', 50) 

# Create a clean view by dropping the heavy text columns just for this preview
# We keep 'jobUrl' this time but truncate it via display options if needed, 
# or drop it if it's still too messy. Let's drop description/html for clarity.
cols_to_hide = ['description', 'descriptionHtml', 'companyUrl', 'jobUrl']
df_display = df.drop(columns=cols_to_hide, errors='ignore')

# USE DISPLAY() INSTEAD OF PRINT()
# This renders a beautiful HTML table in Jupyter Notebooks
display(df_display.sample(5))

print(f"\n✅ Data Check Complete. Total Records: {len(df)}")

🔍 Starting Data Quality Check...

--- 📊 Job Role Distribution ---
role
SOC Technology Specialists                   1181
Direct SOC Analyst                            743
Sales, Management, Training & Consulting      201
Uncategorized                                 116
Incident Response and Threat Intelligence      76
Cyber Defense and Operations                   20
Name: count, dtype: int64

--- 🎲 Random Sample of 5 Jobs (Clean Data Preview) ---


,title,companyName,location,country,region,city,contractType,sector,experienceLevel,source,role,skills
883,EXPERT SECURITE OPERATIONNELLE SI H/F,ICADE,"Puteaux, Île-de-France, France",France,Île-de-France,Puteaux,Full-time,Real Estate,Associate,LinkedIn,SOC Technology Specialists,"{'Technical Skills': ['SIEM', 'EDR', 'CEH', 'C..."
1096,Senior Consultant - Technology Consulting - Cy...,EY,"Renens, Vaud, Switzerland",Switzerland,Vaud,Renens,Full-time,Professional Services,Mid-Senior level,LinkedIn,"Sales, Management, Training & Consulting","{'Programming languages': ['Go'], 'Ability': [..."
84,SOC Security Engineer (m/w/d),Amprion GmbH,"Pulheim, North Rhine-Westphalia, Germany",Germany,North Rhine-Westphalia,Pulheim,Full-time,Utilities,Entry level,LinkedIn,SOC Technology Specialists,"{'Ability': ['Deutsch', 'Berufserfahrung', 'En..."
1870,Młodszy Specjalista ds. Cyberbezpieczeństwa,Kuchnia Vikinga,"Białystok, Podlaskie, Poland",Poland,Podlaskie,Białystok,Full-time,Food and Beverage Manufacturing,Entry level,LinkedIn,Uncategorized,"{'Programming languages': ['PowerShell'], 'Tec..."
309,Sr. Security Incident Handler,Databricks,"London, England, United Kingdom",United Kingdom,England,London,Full-time,Software Development,Mid-Senior level,LinkedIn,Direct SOC Analyst,"{'Technical Skills': ['CISSP'], 'Soft Skills':..."



✅ Data Check Complete. Total Records: 2337


# 6. FINAL EXPORT

In [6]:
# --- 5. FINAL EXPORT ---

try:
    with open(FINAL_OUTPUT, 'w', encoding='utf-8') as f:
        json.dump(processed_jobs, f, indent=4, ensure_ascii=False)
    
    print(f"🎉 Success! The file '{FINAL_OUTPUT}' has been generated.")
    print("This file is now ready for Tableau ingestion.")
    
except Exception as e:
    print(f"❌ Error saving file: {e}")

🎉 Success! The file 'SOC_Analyst_Master_Dataset.json' has been generated.
This file is now ready for Tableau ingestion.
